In [1]:
from bls_py import bls, ec
from random import randint, seed

We will work in the G1 (Fq) group since public keys are in the G1 group.

In [2]:
G = ec.generator_Fq() # G1
l = ec.default_ec.n   # the G1 group order
H = randint(0, l) * G # used in ped. com.: xG + aH, in practice we should have a more secure way of generating H

In [3]:
help(bls.AffinePoint.serialize)

Help on function serialize in module bls_py.ec:

serialize(self)



In [4]:
def Hp(point: bls.AffinePoint) -> bls.AffinePoint:
    """Hash a point to point"""
    return ec.hash_to_point_Fq(point.x.serialize() + point.y.serialize())

def Hs(*xs) -> int:
    """Hash to field, result will be an integer in 0..l"""
    h = b''
    for x in xs:
        if type(x) == str:
            b = bytes(x, encoding="utf-8")
        elif type(x) == bls.AffinePoint:
            b = x.serialize()
        elif type(x) == int:
            b = x.to_bytes(256, "big")
        elif type(x) in [list, tuple]:
            b = b''
            for xi in x:
                b += Hs(xi).to_bytes(256, "big")
        else:
            raise Exception("BAD VALUE", x)

        h = ec.hash256(h + b)

    return int.from_bytes(h, "big") % l
    
def rand_Z():
    return randint(1, l)

def rand_P():
    return rand_Z() * G
    
def commit(value, mask):
    """Pederson Commitment to `value` masked by `mask`"""
    return mask * G + value * H

In [21]:
seed(0)

N = 12
M = 24

msg = "hello"


# Real Signer
x_pi = [rand_Z() for _ in range(M)]
P_pi = [x * G for x in x_pi]

# Decoy Keys
P = [[rand_Z() * G for _ in range(M)] for _ in range(N - 1)]

pi = randint(0, N-1)
print("pi =", pi)

P.insert(pi, P_pi)

# Key Images

I = [x_pi[m] * Hp(P[pi][m]) for m in range(M)]


# Blinding Values

alpha = [rand_Z() for _ in range(M)]
r = [[rand_Z() if i != pi else None for _ in range(M)] for i in range(N)]

# Compute our c's

c = [None for _ in range(N)]

c[(pi + 1) % N] = Hs(
    msg,
    *[alpha[m] * G for m in range(M)],
    *[alpha[m] * Hp(P[pi][m]) for m in range(M)]
)

for offset in range(1, N):
    n = (pi + offset) % N
    c[(n + 1) % N] = Hs(
        msg,
        *[r[n][m] * G + c[n] * P[n][m] for m in range(M)],
        *[r[n][m] * Hp(P[n][m]) + c[n] * I[m] for m in range(M)]
    )
    
# Correct r[pi] using r_pi_j = alpha_j - c_pi * x_pi_j mod l

r[pi] = [(alpha[m] - c[pi] * x_pi[m]) % l for m in range(M)]

# For our sanity, check a few identities
for m in range(M):
    assert x_pi[m] * G == P[pi][m]
    assert (alpha[m] - c[pi]*x_pi[m]) % l * G == alpha[m] * G - (c[pi] * x_pi[m]) * G
    assert (alpha[m] - c[pi]*x_pi[m]) % l * G + c[pi] * P[pi][m] == alpha[m] * G
    assert r[pi][m] * G + c[pi]*P[pi][m] == (alpha[m] - c[pi]*x_pi[m]) % l * G + c[pi] * P[pi][m]
    assert r[pi][m] * Hp(P[pi][m]) + c[pi] * I[m] == (alpha[m] - c[pi]*x_pi[m]) % l * Hp(P[pi][m]) + c[pi] * I[m]
    assert x_pi[m] * Hp(P[pi][m]) == I[m]
    assert -c[pi] * x_pi[m] * Hp(P[pi][m]) == -c[pi] * I[m]
    assert (alpha[m] - c[pi] * x_pi[m]) % l * Hp(P[pi][m]) == alpha[m] * Hp(P[pi][m]) - c[pi] * I[m]
    assert alpha[m] * Hp(P[pi][m]) - c[pi] * I[m] == alpha[m] * Hp(P[pi][m]) - c[pi] * x_pi[m] * Hp(P[pi][m])
    assert r[pi][m] * Hp(P[pi][m]) + c[pi] * I[m] == (alpha[m] - c[pi] * x_pi[m]) % l * Hp(P[pi][m]) + c[pi] * I[m]

sig = (c[0], r, I)

pi = 10


In [23]:
# Verification

for m in range(M):
    assert l * I[m] == 0 * G
    
cprime = [None for _ in range(N)]
cprime[0] = c[0]

for n in range(N):
    cprime[(n + 1) % N] = Hs(
        msg,
        *[r[n][m] * G + cprime[n] * P[n][m] for m in range(M)],
        *[r[n][m] * Hp(P[n][m]) + cprime[n] * I[m] for m in range(M)]
    )
    
assert c[0] == cprime[0]

print("all checks out")

all checks out
